DOCUMENT LOADING AND PROCESSING
Libraries: 
- PDFPlumberLoader: Load and processing PDF document. 
- BeautifulSoup: Processing HTML documents.

PDF PROCESSING:

In [1]:
from langchain_community.document_loaders import PDFPlumberLoader

In [2]:
loader = PDFPlumberLoader("C:/repo/python/ml/projects/agent/Bank_Card_Comparison/data/FILES/Tarifas-de-Tarjetas-de-Debito.pdf")
pdf_docs = []
pdf_docs.extend(loader.load())

HTML Processing:

In [3]:
from bs4 import BeautifulSoup
from langchain.schema import Document

In [4]:
html_path = "C:/repo/python/ml/projects/agent/Bank_Card_Comparison/data/FILES/tarifario-de-tarjetas-de-crdito.html"
html_documents = []
with open(html_path, 'r', encoding='utf-8') as file:
    content = file.read()    
    soup = BeautifulSoup(content, 'html.parser')    
    # Remove script and style elements
    for script in soup(["script", "style"]):
        script.extract()    
    # Get text
    text_content = soup.get_text(separator=' ', strip=True)    
    # Create a Document object instead of a dictionary
    html_documents.append(
        Document(
            page_content=text_content,
            metadata={"source": html_path, "file_type": "html"}
        )
    )

DOCUMENT SPLITTING
Libraries:
- RecursiveCharacterTextSplitter

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
chunk_size = 500
chunk_overlap = 100

In [7]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    # Prioritize splitting on these characters to preserve fee structure
    separators=["\n\n", "\n", ". ", ", ", " ", ""],
    # Keep table rows together
    keep_separator=True
)

In [8]:
all_documents = pdf_docs + html_documents
print (all_documents)

[Document(metadata={'source': 'C:/repo/python/ml/projects/agent/Bank_Card_Comparison/data/FILES/Tarifas-de-Tarjetas-de-Debito.pdf', 'file_path': 'C:/repo/python/ml/projects/agent/Bank_Card_Comparison/data/FILES/Tarifas-de-Tarjetas-de-Debito.pdf', 'page': 0, 'total_pages': 4, 'CreationDate': "D:20250311100139-05'00'", 'Creator': 'Adobe Illustrator 29.1 (Macintosh)', 'CreatorVersion': '21.0.0', 'ModDate': "D:20250311100139-05'00'", 'Producer': 'Adobe PDF library 17.00', 'Title': 'BI_TarifarioTDD_v3'}, page_content='Tarifas de Tarjetas de Débito\nVigente a partir del 15 de noviembre del 2024\nTarjeta de Débito Clave\nTarjeta Clave - Cargos Preferencial Plus Personal\nAnualidad de tarjeta Clave\nUSD 35.00(1) USD 35.00 USD 35.00\ninstantánea\nAnualidad de tarjeta Clave\nUSD 40.00(1) USD 40.00 USD 40.00\npersonalizada\nCargo de Tarjeta vinculada a USD 1.66 USD 1.66 USD 1.66\nCuenta Express ( Mensual ) ( Mensual ) ( Mensual )\nCargo por reemplazo de tarjeta\nUSD 20.00 USD 20.00 USD 20.00\nper

In [9]:
splits = text_splitter.split_documents(all_documents)

In [10]:
for i, split in enumerate(splits):
    print(f"Split {i+1}:")
    print(split)
    print("\n" + "="*40 + "\n")

Split 1:
page_content='Tarifas de Tarjetas de Débito
Vigente a partir del 15 de noviembre del 2024
Tarjeta de Débito Clave
Tarjeta Clave - Cargos Preferencial Plus Personal
Anualidad de tarjeta Clave
USD 35.00(1) USD 35.00 USD 35.00
instantánea
Anualidad de tarjeta Clave
USD 40.00(1) USD 40.00 USD 40.00
personalizada
Cargo de Tarjeta vinculada a USD 1.66 USD 1.66 USD 1.66
Cuenta Express ( Mensual ) ( Mensual ) ( Mensual )
Cargo por reemplazo de tarjeta
USD 20.00 USD 20.00 USD 20.00
perdida o robada(2)' metadata={'source': 'C:/repo/python/ml/projects/agent/Bank_Card_Comparison/data/FILES/Tarifas-de-Tarjetas-de-Debito.pdf', 'file_path': 'C:/repo/python/ml/projects/agent/Bank_Card_Comparison/data/FILES/Tarifas-de-Tarjetas-de-Debito.pdf', 'page': 0, 'total_pages': 4, 'CreationDate': "D:20250311100139-05'00'", 'Creator': 'Adobe Illustrator 29.1 (Macintosh)', 'CreatorVersion': '21.0.0', 'ModDate': "D:20250311100139-05'00'", 'Producer': 'Adobe PDF library 17.00', 'Title': 'BI_TarifarioTDD_v3'

VECTOR STORE AND EMBEDDING
Libraries:
- OllamaEmbeddings
- Chroma


In [1]:
from langchain_ollama import OllamaEmbeddings
from langchain.vectorstores import Chroma
import os

In [2]:
# Set up Ollama embedding model
OllamaEmbeddingModel = OllamaEmbeddings(
    model="nomic-embed-text:latest", ##ollama
    base_url="http://localhost:11434",
)

In [3]:
#persist_directory =   "./data/CHROMADB"
persist_directory = "C:/repo/python/ml/projects/agent/Bank_Products_Expert/data/vector_store"

In [4]:
vectordb = None
if os.path.exists(persist_directory):    
    vectordb = Chroma(
        persist_directory=persist_directory,
        embedding_function=OllamaEmbeddingModel
    )
    print("Loaded existing banking database")
else:
    vectordb = Chroma.from_documents(
                documents=splits,
                embedding=OllamaEmbeddingModel,
                persist_directory=persist_directory
            )
    print("Creating new banking database")


C:\Users\eucli\AppData\Local\Temp\ipykernel_16504\2410241417.py:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(


Loaded existing banking database


In [5]:
print(vectordb._collection.count())

1760


In [19]:
question = "Listame los productos de banco general"
docs = vectordb.similarity_search(question,k=20)
for doc in docs:
    print(doc.page_content)
    print("\n" + "="*40 + "\n")

, comisiones y recargos Guía rápida de marca Contáctenos Síguenos Enlaces importantes Yappy ProFuturo General de Seguros Vale General Fundación Sus Buenos Vecinos Oferta de Bienes En Exclusiva Empresa General de Inversiones Grupo Financiero BG Enlaces importantes Yappy ProFuturo General de Seguros Vale General Fundación Sus Buenos Vecinos Oferta de Bienes En Exclusiva Empresa General de Inversiones Grupo Financiero BG Síguenos Copyright © Banco General


, comisiones y recargos Guía rápida de marca Contáctenos Acerca de Banco General Información Corporativa Corporate Information Trabaja con nosotros Privacidad y Seguridad Actualización de datos Tasas


Cargos por cuentas de terceros
Gastos por Notaría, gastos legales y administrativos de la Solicitud o cambio de Beneficiario. (No se graba ITBMS).
B/. 10.00
(*) Este producto no esta en ofrecimiento.
(**) Este producto solamente aplica para apertura por Banca en Línea.
(***) Las tasas de interes puede variar según las condiciones del pro

In [13]:
len(docs)

3

In [21]:
docs = vectordb.max_marginal_relevance_search(question,k=20, fetch_k=20)
for doc in docs:
    print(doc.page_content)
    print("\n" + "="*40 + "\n")

, comisiones y recargos Guía rápida de marca Contáctenos Síguenos Enlaces importantes Yappy ProFuturo General de Seguros Vale General Fundación Sus Buenos Vecinos Oferta de Bienes En Exclusiva Empresa General de Inversiones Grupo Financiero BG Enlaces importantes Yappy ProFuturo General de Seguros Vale General Fundación Sus Buenos Vecinos Oferta de Bienes En Exclusiva Empresa General de Inversiones Grupo Financiero BG Síguenos Copyright © Banco General


, comisiones y recargos Guía rápida de marca Contáctenos Acerca de Banco General Información Corporativa Corporate Information Trabaja con nosotros Privacidad y Seguridad Actualización de datos Tasas


Cargos por cuentas de terceros
Gastos por Notaría, gastos legales y administrativos de la Solicitud o cambio de Beneficiario. (No se graba ITBMS).
B/. 10.00
(*) Este producto no esta en ofrecimiento.
(**) Este producto solamente aplica para apertura por Banca en Línea.
(***) Las tasas de interes puede variar según las condiciones del pro

In [ ]:
####vectordb.persist()

C:\Users\eucli\AppData\Local\Temp\ipykernel_18356\3711397106.py:1: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


DOCUMENT RETRIEVAL
Libraries:
- 

In [10]:
def pretty_print_docs(docs):
    print(f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]))

In [11]:
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo
from langchain_community.llms import Ollama

In [12]:
metadata_field_info = [
    AttributeInfo(
        name="Country",
        description="Panama.",
        type="string",
    ),     
    AttributeInfo(
        name="bank",
        description="Bank name associated with products or services. Should one of the following: 'Banco General','Towerbank','PacificBank','BAC','BancoDavivienda','Canalbank','UniBank','BICSA','Bancolombia','BancoAliado','BancoLafisePanamá','St.GeorgesBank','Terrabank','Itaú','BancoBCT','Multibank','Banistmo','MercantilBanco','BancoAzteca','BancodeBogotá','BancoDelta','BancoFicohsa','BancoGeneral','BancoLaHipotecaria','BancoPichincha','BancoPrival','Banesco','Banisi','BankofChinaLimited','BBPBank','Bi-Bank','BLADEX','Citibank','CredicorpBank','FPBBank','GlobalBank','ICBC','KebHanaBank','MegaBank','Metrobank','MMGBank','Scotiabank','ASBBank','AustrobankOverseas','BancoCreditAndorra','BancoDaviviendaInternacional','BancodeBogota','BancodeCreditodelPeru','BancodeOccidente','BHDInternationalBank','BPRBANK','GNBSudamerisBank','InteligoBank','ITAU','PopularBank','TAGBank','BancoNacionaldePanamá','BancodeDesarrolloAgropecuario(BDA)','BancoHipotecarioNacional(BHN)'",
        type="string",
    ),
    AttributeInfo(
        name="products and services",
        description="Products or services offered by the bank. Should one of the following: 'Tarjeta de Débito','Tarjeta de Crédito','Cuenta de Ahorros','Cuenta Corriente','Préstamos Personales','Préstamos Hipotecarios','Inversiones','Seguros','Banca por Internet','Banca Móvil','Transferencias Internacionales','Cajeros Automáticos','Atención al Cliente','Cheques','Transferencias','Cuentas'",
        type="string",
    ),
    AttributeInfo(
        name="actions and fees",
        description="Key actions or fees associated with the products or services. Should one of the following: 'Apertura','Cancelación,'Membresía,'Comisión,'Cargo','ITBMS','Mora','Desembolso','Renovación','Reemplazo,'Retiro de Efectivo','Transferencia Nacional','Transferencia Internacional','Pago de Servicios','Comisión por Uso de Cajero Automático','Intereses por Préstamos','Intereses por Tarjeta de Crédito','Seguro de Protección de Pagos','Cargo por Cheque Devuelto','Cargo por Sobregiro','Cargo por Inactividad'",
        type="string",
    ),
    AttributeInfo(
        name="customer type",
        description="Type of customer entities. Should one of the following: 'Personas Naturales','Personas Jurídicas','Empresas','Instituciones','Organizaciones','Clientes Individuales','Clientes Corporativos'",
        type="string",
    ),        
]

In [17]:
document_content_description = "Documents contain fee schedule of Panamanian banks"
llm = Ollama(model="gemma3:1b", temperature=0.3, base_url="http://localhost:11434")
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectordb,
    document_content_description,
    metadata_field_info,
    verbose=True
)

In [22]:
question = "Listame los productos de banco general"

In [23]:
docs = retriever.get_relevant_documents(question)
pretty_print_docs(docs)

Document 1:

, comisiones y recargos Guía rápida de marca Contáctenos Síguenos Enlaces importantes Yappy ProFuturo General de Seguros Vale General Fundación Sus Buenos Vecinos Oferta de Bienes En Exclusiva Empresa General de Inversiones Grupo Financiero BG Enlaces importantes Yappy ProFuturo General de Seguros Vale General Fundación Sus Buenos Vecinos Oferta de Bienes En Exclusiva Empresa General de Inversiones Grupo Financiero BG Síguenos Copyright © Banco General
----------------------------------------------------------------------------------------------------
Document 2:

, comisiones y recargos Guía rápida de marca Contáctenos Acerca de Banco General Información Corporativa Corporate Information Trabaja con nosotros Privacidad y Seguridad Actualización de datos Tasas
----------------------------------------------------------------------------------------------------
Document 3:

por el Banco Corresponsal y/o el Banco del Beneficiario Queda entendido que las tasas y comisiones pac

In [16]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

In [22]:
compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=vectordb.as_retriever()
)

In [23]:
compressed_docs = compression_retriever.get_relevant_documents(question)
pretty_print_docs(compressed_docs)

C:\Users\eucli\AppData\Local\Temp\ipykernel_15224\773383336.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  compressed_docs = compression_retriever.get_relevant_documents(question)


Document 1:

N°3 DOCUMENTOS REQUERIDOS PARA EMPRESAS CONSTITUIDAS EN ECUADOR.
ADEMÁS DE LOS DOCUMENTOS INDICADOS EN N1.
Escritura de constitución.
Enmienda a la escritura de constitución (en caso de existir).
RUC (actualizado).
Cumplimiento de obligaciones (actualizado).
Nombramiento de los directores actualizados o vigentes.
Nómina de accionistas: Adjuntar copias de cédula si los directores son personas naturales. En caso de ser personas
----------------------------------------------------------------------------------------------------
Document 2:

Entrega de un libro recordatorio con aspectos relacionados a la asistencia de las honras. Espacio por 3 años en arrendamiento en cementerio municipal a nivel nacional. Proveer al cliente de cofre de corte lineal en caso de inhumación o urna en caso de cremación.
----------------------------------------------------------------------------------------------------
Document 3:

Aviso de operación.
Referencias comerciales.
Referencias bancarias

QUESTION ANSWERING
Libraries:
-

In [25]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

In [26]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [27]:
# Build prompt
template = """You have to answer in spanish. Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Keep the answer as concise as possible. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

In [45]:
question = "Menciona todos los nombres de los bancos que se mencionan en el documento"

In [37]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [46]:
result = qa_chain({"query": question})
result["result"]

'No sé los nombres de los bancos mencionados en el documento.'

In [31]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="map_reduce"
)

In [32]:
result = qa_chain_mr({"query": question})
result["result"]

c:\Users\eucli\anaconda3\envs\AGENTS_ENV\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
Token indices sequence length is longer than the specified maximum sequence length for this model (1576 > 1024). Running this sequence through the model will result in indexing errors


'cargas y comisiones - Banco Nacional de Panamá, cargas y comisiones - Banco Nacional de Panamá, cargas y comisiones - Banco Nacional de Panamá'

In [33]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="refine"
)

In [47]:
result = qa_chain_mr({"query": question})
result["result"]

'Pacific Bank\n'

In [5]:
from pathlib import Path
path = Path("./data/CHROMADB")
print(path.exists)

<bound method Path.exists of WindowsPath('data/CHROMADB')>
